Read file

In [40]:
import requests
import os
import warnings
from tqdm import tqdm
import numpy as np
import zipfile
import rioxarray
import xarray_leaflet
from ipyleaflet import Map, basemaps, LayersControl, WidgetControl
from ipywidgets import FloatSlider
import matplotlib.pyplot as plt
import fsspec
import s3fs
import xarray as xr
store = fsspec.get_mapper('https://storage.sbg.cloud.ovh.net/v1/AUTH_d40770b0914c46bfb19434ae3e97ae19/hdsa-public/prisma-subset.zarr')
dataset = xr.open_zarr(store) # open with xarray
rf = dataset['reflectance'] # First we load the reflectance variable


In [32]:
rf

,Array,Chunk
Bytes,677.38 MiB,128.00 kiB
Shape,"(234, 1221, 1243)","(1, 256, 256)"
Count,5851 Tasks,5850 Chunks
Type,uint16,numpy.ndarray


Transform to EPSG 4326

In [41]:
from pyproj import Transformer

r3 = rf.isel(wavelength=0,latitude=slice(0,1000),longitude=slice(0,1000))

# Retrieve dataset and coordinates

lat = r3.longitude.data
lon = r3.latitude.data

# Reproject coordinates
transformer = Transformer.from_crs("epsg:32630", "epsg:4326") # UTM30 to EPSG 4326
latlon_reprojected = transformer.transform(lon,lat) #Reproject lat lon
new_lon = latlon_reprojected[0]
new_lat = latlon_reprojected[1]

# Assign reprojected coordinates
r3 = r3.assign_coords(latitude=new_lat)
r3 = r3.assign_coords(longitude=new_lon)
r3 = r3.assign_coords({"longitude": new_lon, "latitude": new_lat})

In [35]:
r3

<xarray.DataArray 'reflectance' (latitude: 1000, longitude: 1000)>
dask.array<getitem, shape=(1000, 1000), dtype=uint16, chunksize=(256, 256), chunktype=numpy.ndarray>
Coordinates:
  * latitude    (latitude) float64 30.1 30.1 30.1 30.1 ... 30.33 30.33 30.33
  * longitude   (longitude) float64 1.875 1.875 1.875 ... 2.096 2.096 2.096
    wavelength  float64 402.4
Attributes:
    addoffset:    0.0
    scalefactor:  1.5259021896696422e-05
    units:        reflectance

In [43]:
#r3.rio.set_spatial_dims("longitude","latitude",inplace=True)
r3 = r3.swap_dims({"longitude":"x","latitude":"y"})
r3.rio.write_crs("EPSG:4326",inplace=True)
r3.rio.write_nodata(0, inplace=True)


<xarray.DataArray 'reflectance' (y: 1000, x: 1000)>
dask.array<getitem, shape=(1000, 1000), dtype=uint16, chunksize=(256, 256), chunktype=numpy.ndarray>
Coordinates:
    latitude     (y) float64 30.1 30.1 30.1 30.1 ... 30.33 30.33 30.33 30.33
    longitude    (x) float64 1.875 1.875 1.875 1.876 ... 2.096 2.096 2.096 2.096
    wavelength   float64 402.4
    spatial_ref  int64 0
Dimensions without coordinates: y, x
Attributes:
    addoffset:    0.0
    scalefactor:  1.5259021896696422e-05
    units:        reflectance
    _FillValue:   0

In [44]:
m = Map(center=[40, -6], zoom=4, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')
m


Map(center=[40, -6], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [45]:
warnings.filterwarnings("ignore")
r3.leaflet.plot(m, colormap=plt.cm.terrain)


Url()

LocalTileLayer(name='reflectance', options=['attribution', 'detect_retina', 'max_native_zoom', 'max_zoom', 'mi…